In [13]:
import pandas as pd
import numpy as np
import random
import os
import sys
import requests
import time
import datetime as dt

from dotenv import load_dotenv
from flipside import Flipside
from prophet import Prophet

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from scripts.utils import flipside_api_results, set_random_seed
from sql_queries.sql import Optimistic_Domains_Sales, eth_price

from scripts.apis import alchemy_metadata_api, fetch_event_type, clean_data, fetch_all_events, hourly

In [14]:
pd.options.display.float_format = '{:,.2f}'.format

In [15]:
load_dotenv()

seed = 20
set_random_seed(seed)

In [16]:
flipside_api_key = os.getenv('FLIPSIDE_API_KEY')
alchemy_api_key = os.getenv('ALCHEMY_API_KEY')
opensea_api_key = os.getenv('OPENSEA_API_KEY')

print(alchemy_api_key)

6AUlaGmWe505S7gRPZXVh4YEFgJdYHy5


# Data Sources

## Alchemy

In [17]:
optimistic_domains = alchemy_metadata_api(alchemy_api_key, 'optimism', '0xC16aCAdf99E4540E6f4E6Da816fd6D2A2C6E1d4F')

Base URL: https://opt-mainnet.g.alchemy.com/nft/v3/6AUlaGmWe505S7gRPZXVh4YEFgJdYHy5/getNFTsForContract
Number added: 100 | Total number of NFTs: 100, Next page key: 0x0000000000000000000000000000000000000000000000000000000000000064
Number added: 100 | Total number of NFTs: 200, Next page key: 0x00000000000000000000000000000000000000000000000000000000000000c8
Number added: 100 | Total number of NFTs: 300, Next page key: 0x000000000000000000000000000000000000000000000000000000000000012c
Number added: 100 | Total number of NFTs: 400, Next page key: 0x0000000000000000000000000000000000000000000000000000000000000190
Number added: 100 | Total number of NFTs: 500, Next page key: 0x00000000000000000000000000000000000000000000000000000000000001f4
Number added: 100 | Total number of NFTs: 600, Next page key: 0x0000000000000000000000000000000000000000000000000000000000000258
Number added: 100 | Total number of NFTs: 700, Next page key: 0x00000000000000000000000000000000000000000000000000000000000

## Opensea

In [18]:
Three_DNS_data = fetch_all_events(api_key=opensea_api_key,collection='3dns-powered-domains')
Three_DNS_data = Three_DNS_data[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]

ens_data = fetch_all_events(api_key=opensea_api_key,collection='ens')
ens_data = ens_data[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]

optimism_name_service_data = fetch_all_events(api_key=opensea_api_key,collection='optimism-name-service')
optimism_name_service_data = optimism_name_service_data[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]

unstoppable_sales_data = fetch_all_events(api_key=opensea_api_key,collection='unstoppable-domains')
unstoppable_sales_data = unstoppable_sales_data[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]

base_domains_metadata_pd = fetch_all_events(api_key=opensea_api_key,collection='basedomainnames')
base_domains_metadata_pd = base_domains_metadata_pd[['dt','token_symbol','token_amt_clean','nft_identifier','nft_name']]


Fetching sale: Page 1, Events Fetched: 50, Total Events: 50, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA3LTI1KzEyJTNBNTElM0EzOSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0yNDY3NjM1MjEzOQ==
Fetching sale: Page 2, Events Fetched: 50, Total Events: 100, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA1LTA4KzA4JTNBMjklM0EzNSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0yMjQwNDY0NTU4MQ==
Fetching sale: Page 3, Events Fetched: 50, Total Events: 150, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTAyLTEwKzE5JTNBMzQlM0E0NSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0yMDA2NTI1NDc4OQ==
Fetching sale: Page 4, Events Fetched: 15, Total Events: 165, next cursor: 
Total events fetched: 165
Fetching sale: Page 1, Events Fetched: 50, Total Events: 50, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA4LTA3KzE1JTNBNTUlM0E1OSYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1waz0yNTA3Njc4MzE4NA==
Fetching sale: Page 2, Events Fetched: 50, Total Events: 100, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA4LTA1KzIzJTNBNTYlM0E0NyYtZXZlbnRfdHlwZT1zdWNjZXNzZnVsJi1wa

KeyboardInterrupt: 

## Flipside

In [ ]:
flipside = Flipside(flipside_api_key, "https://api-v2.flipsidecrypto.xyz")

In [ ]:
optimistic_domains_sales = flipside_api_results(Optimistic_Domains_Sales, flipside_api_key)

In [ ]:
prices_data = flipside_api_results(eth_price, flipside_api_key)

### Optimistic Domains Data Engineering

In [ ]:
# optimistic_domains_sales = optimistic_domains_sales.dropna(subset=['TOKENID'])
# optimistic_domains_sales['TOKENID']
# optimistic_domains_sales['TOKENID'] = optimistic_domains_sales['TOKENID'].astype(int)
optimistic_domains_sales.rename(columns={"tokenid":"tokenId"}, inplace=True)
optimistic_data = pd.merge(optimistic_domains_sales, optimistic_domains, on='tokenId', how='left')
optimistic_data.rename(columns={"tokenId":"nft_identifier","name":"nft_name", "day":"dt"}, inplace=True)
optimistic_data

,dt,nft_identifier,price,__row_index,nft_name
0,2022-10-28T09:00:00.000Z,1069,0.02,0,aave.op
1,2023-02-11T03:00:00.000Z,347,0.00,1,lo.op
2,2023-02-18T19:00:00.000Z,731,0.03,2,74.op
3,2023-02-20T21:00:00.000Z,413,0.03,3,dip.op
4,2023-02-20T21:00:00.000Z,1076,0.05,4,qx.op
5,2022-06-19T00:00:00.000Z,787,0.02,5,188.op
6,2022-06-03T04:00:00.000Z,482,0.01,6,luxury.op
7,2022-06-04T02:00:00.000Z,550,0.01,7,princess.op
8,2022-06-04T02:00:00.000Z,549,0.05,8,prince.op
9,2022-08-04T20:00:00.000Z,933,0.01,9,tro.op


In [ ]:
domain_path = 'data/domain-name-sales.tsv'  
domain_data = pd.read_csv(domain_path, delimiter='\t')
domain_data.set_index('date', inplace=True)
domain_data = domain_data.drop(columns=['venue'])
domain_data.sort_index(inplace=True)
domain_data

domain_data = domain_data.reset_index()
domain_data = domain_data.rename(columns={"date":"dt","price":"price_usd"})
domain_data['dt'] = pd.to_datetime(domain_data['dt'])
domain_data['dt'] = domain_data['dt'].dt.tz_localize('UTC')
domain_data['dt'] = pd.to_datetime(domain_data['dt'])
domain_data

### Rest of Data Eningeering

In [ ]:
prices_data = prices_data.dropna()
prices_data['symbol'] = prices_data['symbol'].replace('WETH', 'ETH')


prices_data = prices_data.pivot(index='dt',columns='symbol',values='price')
prices_data = prices_data.reset_index()

prices_data['dt'] = pd.to_datetime(prices_data['dt'])
prices_data

KeyError: 'symbol'

In [ ]:
Three_DNS_data = hourly(Three_DNS_data)
optimism_name_service_data = hourly(optimism_name_service_data)
ens_data = hourly(ens_data)
unstoppable_sales_data = hourly(unstoppable_sales_data)
base_domains_metadata_pd = hourly(base_domains_metadata_pd)

Three_DNS_data

In [ ]:
Three_DNS_data = Three_DNS_data.merge(prices_data, how='left', on='dt')
Three_DNS_data['price_usd'] = Three_DNS_data['token_amt_clean'] * Three_DNS_data['ETH']
Three_DNS_data

optimism_name_service_data = optimism_name_service_data.merge(prices_data, how='left', on='dt')
optimism_name_service_data['price_usd'] = optimism_name_service_data['token_amt_clean'] * optimism_name_service_data['ETH']
optimism_name_service_data

ens_data = ens_data.merge(prices_data, how='left', on='dt')
ens_data['price_usd'] = ens_data['token_amt_clean'] * ens_data['ETH']
ens_data

unstoppable_sales_data = unstoppable_sales_data.merge(prices_data, how='left', on='dt')
unstoppable_sales_data['price_usd'] = unstoppable_sales_data['token_amt_clean'] * unstoppable_sales_data['ETH']
unstoppable_sales_data

base_domains_metadata_pd = base_domains_metadata_pd.merge(prices_data, how='left', on='dt')
base_domains_metadata_pd['price_usd'] = base_domains_metadata_pd['token_amt_clean'] * base_domains_metadata_pd['ETH']
base_domains_metadata_pd

In [ ]:
optimistic_data.rename(columns={'DAY':'dt','PRICE_USD':'price_usd','PRICE':'token_amt_clean'}, inplace=True)

In [ ]:
optimistic_data['dt'] = pd.to_datetime(optimistic_data['dt'])
optimistic_data['dt'] = optimistic_data['dt'].dt.tz_localize('UTC')
optimistic_data['dt'] = pd.to_datetime(optimistic_data['dt'])


In [ ]:
combined_dataset = pd.concat([
    ens_data[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    optimistic_data[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    optimism_name_service_data[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    unstoppable_sales_data[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    base_domains_metadata_pd[['dt','nft_name','price_usd','token_amt_clean']].dropna(),
    Three_DNS_data[['dt','nft_name','price_usd','token_amt_clean']].dropna()
], ignore_index=True)

combined_dataset = combined_dataset.drop_duplicates()
combined_dataset['dt'] = pd.to_datetime(combined_dataset['dt'], errors='coerce')
combined_dataset = combined_dataset.sort_values(by='dt')
combined_dataset = combined_dataset.reset_index(drop=True)
combined_dataset
